# Create centerline road network 
Based on BGT data

In [ ]:
# Select where to run notebook: "azure" or "local"
my_run = "azure"

In [ ]:
import set_path

import numpy as np
import pandas as pd

import shapely
import shapely.ops as so
import geopandas as gpd
from geopandas import GeoDataFrame
from centerline.geometry import Centerline
from tqdm.notebook import tqdm
tqdm.pandas()

import bgt_utils
import plot_utils

import folium

import settings as st
import functions as f

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

In [ ]:
os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

## Import BGT data

In [ ]:
# Get BGT data
print('Pulling BGT data...')
df_bgt_full = bgt_utils.get_bgt_data_for_bbox(st.bbox, st.bgt_road_layers)
df_bgt_full['naam'].value_counts()
print('Pulled BGT data.')

## Select pilot areas

In [ ]:
# Import areas
df_areas = gpd.read_file(cf.output_pilot_area)

In [ ]:
# Only keep BGT data within pilot areas
df_bgt = df_bgt_full.sjoin(df_areas, how='inner', predicate='intersects')  # note: only road polygons fully inside area are included
df_bgt['naam_left'].value_counts()

## Pre-process BGT data

In [ ]:
# Merge road polygons
df_bgt = GeoDataFrame(geometry=gpd.GeoSeries(df_bgt['geometry'].unary_union))
df_bgt = gpd.GeoDataFrame(df_bgt.geometry.explode(index_parts=True))

## Create centerlines

In [ ]:
# Calculate centerlines
df_bgt['centerlines'] = df_bgt.progress_apply(
   lambda row: Centerline(row.geometry, interpolation_distance=0.5), axis=1)

df_bgt['geometry'] = df_bgt['centerlines']
df_bgt = df_bgt.drop(columns=['centerlines'])

In [ ]:
# Merge lines
idxs = df_bgt.index[df_bgt.geom_type != 'LineString'].tolist()
df_bgt.loc[idxs, 'geometry'] = df_bgt.loc[idxs, 'geometry'].progress_apply(so.linemerge)

# Get each centerline in separate row
df_bgt_exp = df_bgt.explode(ignore_index=True)
df_bgt_exp.reset_index()
df_bgt_exp['road_part_id'] = df_bgt_exp.index
df_bgt_exp = df_bgt_exp.set_crs(crs=st.CRS)

## Store final output

In [ ]:
# Write road network to file
df_bgt_exp.to_file(cf.output_road_network, driver='GPKG')

### Visualize

In [ ]:
# set True for satelite background, False for regular background
satelite = False

# Create Folium map
map = folium.Map(
    location=[52.389164, 4.908453], tiles=plot_utils.generate_map_params(satelite=satelite),
    min_zoom=10, max_zoom=25, zoom_start=15,
    zoom_control=True, control_scale=True, control=False
    )

# Add network and new edges
geo_j = folium.GeoJson(df_bgt_exp, style_function=lambda x: {"color": "black", "weight": 2}).add_to(map)

In [ ]:
# Add title
loc = 'Road network'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)

map.get_root().html.add_child(folium.Element(title_html))

In [ ]:
map